In [39]:
import os
import time
import datetime
import pandas as pd

# 根据（spark）Markov.java统计出的马尔可夫模型(model.txt)，对validate.txt中的数据进行预测什么时间应该发出营销邮件
user_action = {}
model = []

#9大状态
states = ["SL", "SE", "SG", "ML", "ME", "MG", "LL", "LE", "LG"]

validate_path = os.path.join(os.getcwd(), "validate.txt")
model_path = os.path.join(os.getcwd(), "model.txt")

#读取validate data，user_action中保存{user_id:[[date1, amount1], [date2, amount2],...]}
validate_data = pd.read_csv(validate_path, sep=',', header=None)
for row_index in validate_data.itertuples():
    user_id = getattr(row_index,'_1')
    date = getattr(row_index, '_3')
    amount = getattr(row_index, '_4')

    if user_id in user_action.keys():
        hist = user_action[user_id]
        lst = [date, amount]
        hist.append(lst)
    else:
        hist = []
        hist.append([date, amount])
        user_action[user_id] = hist
print(user_action)

#读取model data
with open(model_path, 'r', encoding='utf-8') as f_read:
    for line in f_read:
        items = line.strip().split()
        row = []
        for item in items:
            row.append(float(item))
        model.append(row)
print(model)

#根据最近客户的行为数据(至少两次交易)make prediciton
for user_id,user_action_list in user_action.items():
    if len(user_action_list) < 2:
        continue
    state_sequence = []
    last_date = '' 
    prior = user_action_list[0]
    for i in range(1, len(user_action_list)):
        current = user_action_list[i]
        prior_date = prior[0]
        current_date = current[0]
        
        #相隔天数
        prior_date = time.strptime(prior_date, '%Y-%m-%d')
        current_date = time.strptime(current_date, '%Y-%m-%d')
        prior_date = datetime.datetime(prior_date[0], prior_date[1], prior_date[2])
        current_date = datetime.datetime(current_date[0], current_date[1], current_date[2])
        days_diff = (current_date - prior_date).days
        
        dd = 'L'
        if days_diff < 30:
            dd = 'S'
        elif days_diff < 60:
            dd = 'M'
        
        #相差金额
        prior_amount = int(prior[1])
        current_amount = int(current[1])
        
        ad = 'G'
        if prior_amount < 0.9 * current_amount:
            ad = 'L'
        elif prior_amount < 1.1 * current_amount:
            ad = 'E'
            
        state_sequence.append(dd+ad)
        
        prior = current
        last_date = current_date
        
    if state_sequence:
        #根据最近一个状态发送营销邮件日期
        last_state = state_sequence[-1]
        row_index = states.index(last_state)
        row_value = model[row_index] #转移矩阵中行号为row_index的这一行值
        max_value = max(row_value) #row_value中最大值 
        col_index = row_value.index(max_value) #max_value的索引号
        next_state = states[col_index]
        
        if next_state.startswith('S'):
            next_date = last_date + datetime.timedelta(15)
        elif next_state.startswith('E'):
            next_date = last_date + datetime.timedelta(45)
        else:
            next_date = last_date + datetime.timedelta(90)
    
    print('用户：{}, 预测下次发送时间：{}'.format(user_id, next_date))
        

{'user1': [['2020-01-01', 98], ['2020-01-07', 90]], 'user2': [['2020-01-01', 53], ['2020-02-01', 43]], 'user3': [['2020-01-01', 164], ['2020-01-10', 64]], 'user4': [['2020-01-01', 204], ['2020-03-21', 74]], 'user5': [['2020-01-01', 83], ['2020-01-15', 183]], 'user6': [['2020-01-01', 116], ['2020-01-01', 106]]}
[[4.396976638863118e-06, 4.396976638863118e-06, 0.8062208425486636, 0.15858575643387607, 4.396976638863118e-06, 4.396976638863118e-06, 0.035153828227710626, 4.396976638863118e-06, 4.396976638863118e-06], [0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111], [0.8043794973142799, 3.1671227323401235e-06, 3.1671227323401235e-06, 0.1594804651869869, 3.1671227323401235e-06, 3.1671227323401235e-06, 0.03611153339414209, 3.1671227323401235e-06, 3.1671227323401235e-06], [1.225940909648155e-05, 1.225940909648155e-05, 0.8156797842343999, 1.225940909648155e-05, 1.225